# Assigment 5

**Submission deadlines**:

* last lab before 27.06.2022 

**Points:** Aim to get 12 out of 15+ possible points

All needed data files are on Drive: <https://drive.google.com/drive/folders/1uufpGn46Mwv4oBwajIeOj4rvAK96iaS-?usp=sharing> (or will be soon :) )

## Task 1 (5 points)

Consider the vowel reconstruction task -- i.e. inserting missing vowels (aeuioy) to obtain proper English text. For instance for the input sentence:

<pre>
h m gd smbd hs stln ll m vwls
</pre>

the best result is

<pre>
oh my god somebody has stolen all my vowels
</pre>

In this task both dev and test data come from the two books about Winnie-the-Pooh. You have to train two RNN Language Models on *pooh-train.txt*. For the first model use the code below, for the second choose different hyperparameters (different dropout, smaller number of units or layers, or just do any modification you want). 

The code below is based on
https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html

In [1]:
import torch
from collections import Counter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

SEQUENCE_LENGTH = 15

class PoohDataset(torch.utils.data.Dataset):
    def __init__(self, sequence_length, device):
        txt = open('data/pooh_train.txt').read()
        
        self.words = txt.lower().split() # The text is already tokenized
        
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]
        self.sequence_length = sequence_length
        self.device = device


    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.sequence_length], device=self.device),
            torch.tensor(self.words_indexes[index+1:index+self.sequence_length+1], device=self.device)
        )
        
pooh_dataset = PoohDataset(SEQUENCE_LENGTH, device)

cuda


In [2]:
from torch import nn, optim

class LSTMModel(nn.Module):
    def __init__(self, dataset, device):
        super(LSTMModel, self).__init__()
        self.lstm_size = 512
        self.embedding_dim = 100
        self.num_layers = 2
        self.device = device

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size).to(self.device))
        
model = LSTMModel(pooh_dataset, device) 
model.to(device)

LSTMModel(
  (embedding): Embedding(2548, 100)
  (lstm): LSTM(100, 512, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=512, out_features=2548, bias=True)
)

In [3]:
import numpy as np
from torch.utils.data import DataLoader

batch_size = 512
max_epochs = 30

def train(dataset, model):
    model.train()

    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(max_epochs):
        state_h, state_c = model.init_state(SEQUENCE_LENGTH)
        
        for batch, (x, y) in enumerate(dataloader):
            
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()            

            loss.backward()
            optimizer.step()

        print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [4]:
filename = "pooh_2x512_30ep.model"
try:
    model.load_state_dict(torch.load(filename))
except FileNotFoundError:
    train(pooh_dataset, model)
    torch.save(model.state_dict(), filename)

In [5]:
def predict(dataset, model, text, next_words=15):
    model.eval()

    words = text.split()
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        x = x.to(device)
        
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().cpu().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return ' '.join(words)

# DEMO
speakers = ['pooh', 'piglet', 'christopher robin', 'rabbit', 'owl', 'tigger', 'eeyore']
for s in speakers:
    prompt = 'in the morning ' + s 
    for i in range(1):
        print(predict(pooh_dataset, model, prompt, 50))

in the morning pooh sound 's very happy thing -- tiggers could n't get down , but somebody you bounced to balloons too . they could have very flying , but nobody seemed . there is another good song . some can find which some come . which does n't shout no any more
in the morning piglet hearthrug . because having love him , piglet slipped a pooh behind , some help . `` oh , christopher you , tigger 's perhaps '' began piglet , so that anyhow it was n't . `` come along , '' said tigger . chapter became very thoughtful , because
in the morning christopher robin and out of the hundred river which came up behind a left , and now down to make glad cage what happened to christopher robin about the blue door . he looked at his own ceiling which he had left without this in . `` ha , '' said pooh
in the morning rabbit christopher robin would do it coming and strong and roo ran before ( or ready of string when we are quite sure we took christopher robin 's helping matter , and before they all wer

In [6]:
# You can use the code if you want

from collections import defaultdict as dd

vowels = set("aoiuye'")
def devowelize(s):
    rv = ''.join(a for a in s if a not in vowels)
    if rv:
        return rv
    return '_' # Symbol for words without consonants

pooh_words = set(open('data/pooh_words.txt').read().split())
representation = dd(set)

for w in pooh_words:
    r = devowelize(w)
    representation[r].add(w)
    
hard_words = set()
for r, ws in representation.items():
    if len(ws) > 1:
        hard_words.update(ws)
        
print (len(hard_words))    

863


In [7]:
import torch.nn.functional as F

In [8]:
text = "if you happen to have read another book about christopher robin , you may remember that he once had a swan"
words = text.split()
state_h, state_c = model.init_state(len(words))
x = torch.tensor([[pooh_dataset.word_to_index[w] for w in words]], device=device)
next_words = 25
for i in range(next_words):
    y_pred, (new_state_h, new_state_c) = model(x, (state_h, state_c))
    y_pred = y_pred[:, -1:].contiguous()
    state_h, state_c = new_state_h[:, -1:, ...].contiguous(), new_state_c[:, -1:, ...].contiguous()
    x = torch.multinomial(F.softmax(y_pred, dim=-1).flatten(), 1).reshape(1, -1)
    words.append(pooh_dataset.index_to_word[x.item()])
print(" ".join(words))

if you happen to have read another book about christopher robin , you may remember that he once had a swan and then you 'll take together for -- but what happened that nobody says everything down as this ? '' `` well , '' said


In [9]:
def predict(dataset, model, text, next_words=15):
    model.eval()

    words = text.split()
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        x = x.to(device)

        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().cpu().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return ' '.join(words)

In [ ]:
start = "pooh"
words = text.split()

model.eval()
corrected = []
state_h, state_c = model.init_state(1)
x = torch.tensor([[pooh_dataset.word_to_index[start]]], device=device)
for word in words:
    possible_idxs = torch.tensor([pooh_dataset.word_to_index[k] for k in representation[word] if k in pooh_dataset.word_to_index])
    y_pred, (new_state_h, new_state_c) = model(x, (state_h, state_c))
    y_pred = y_pred[:, -1:].contiguous()
    state_h, state_c = new_state_h[:, -1:, ...].contiguous(), new_state_c[:, -1:, ...].contiguous()
    selected = torch.multinomial(F.softmax(y_pred.flatten()[possible_idxs] / T), 1)
    x = possible_idxs[selected].reshape(1, -1)
    corrected.append(pooh_dataset.index_to_word[x.item()])

In [78]:
from typing import List

def _reconstruct(words: List[str], model, start: str, T: float):
    model.eval()
    corrected = []
    state_h, state_c = model.init_state(1)
    x = torch.tensor([[pooh_dataset.word_to_index[start]]], device=device)
    plog = 0
    for word in words:
        possible_idxs = torch.tensor([pooh_dataset.word_to_index[k] for k in representation[word] if k in pooh_dataset.word_to_index], device=device)
        y_pred, (new_state_h, new_state_c) = model(x, (state_h, state_c))
        y_pred = y_pred[:, -1:].contiguous()
        state_h, state_c = new_state_h[:, -1:, ...].contiguous(), new_state_c[:, -1:, ...].contiguous()
        if possible_idxs.numel():
            preds = F.softmax(y_pred.flatten()[possible_idxs] / T, -1)
        else:
            preds = F.softmax(y_pred.flatten() / T, -1)

        selected = torch.multinomial(preds, 1)

        if possible_idxs.numel():
            x = possible_idxs[selected].reshape(1, -1)
        else:
            x = selected.reshape(1, -1)

        corrected.append(pooh_dataset.index_to_word[x.item()])
        plog += torch.log(preds[selected]).item()
    return " ".join([start] + corrected), plog

def reconstruct(text: List[str], model, T: float, n_iter: int = 10):
    if not text:
        return []
    model.eval()
    max_plog = float("-inf")
    for start in representation[text[0]]:
        for _ in range(n_iter):
            correction, plog = _reconstruct(text[1:], model, start, T)
            if plog > max_plog:
                max_plog = plog
                best_correction = correction
    return best_correction

In [79]:
with open("data/pooh_test.txt", "rt") as f:
    test_tokens = f.read().split()

In [80]:
test_input = list(map(devowelize, test_tokens))

In [85]:
reconstructed = reconstruct(test_input, model, 1)

You can assume that only words from pooh_words.txt can occur in the reconstructed text. For decoding you have two options (choose one, or implement both ang get **+1** bonus point)

1. Sample reconstructed text several times (with quite a low temperature), choose the most likely result.
2. Perform beam search.

Of course in the sampling procedure you should consider only words matching the given consonants.

Report accuracy of your methods (for both language models). The accuracy should be computed by the following function, it should be *greater than 0.25*.


```python
def accuracy(original_sequence, reconstructed_sequence):
    sa = original_sequence
    sb = reconstructed_sequence
    score = len([1 for (a,b) in zip(sa, sb) if a == b])
    return score / len(original_sequence)
```


In [87]:
def accuracy(original_sequence, reconstructed_sequence):
    sa = original_sequence
    sb = reconstructed_sequence
    score = len([1 for (a,b) in zip(sa, sb) if a == b])
    return score / len(original_sequence)

In [90]:
accuracy(test_tokens, reconstructed.split())

0.7969128715397372

## Task 2 (6 points)

This task is about text generation. You have to:

**A**. Create text corpora containing texts with similar vocabulary (for instance books from the same genre, or written by the same author). This corpora should have approximately 1M words. You can consider using the following sources: Project Gutenberg (https://www.gutenberg.org/), Wolne Lektury (https://wolnelektury.pl/), parts of BookCorpus, https://github.com/soskek/bookcorpus, but generally feel free. Texts could be in English, Polish or any other language you know.

**B**. choose the tokenization procedure. It should have two stages:

1. word tokenization (you can use nltk.tokenize.word_tokenize, tokenizer from spaCy, pytorch, keras, ...). Test your tokenizer on your corpora, and look at a set of tokens containing both letters and special characters. If some of them should be in your opinion treated as a sequence of tokens, then modify the tokenization procedure

2. sub-word tokenization (you can either use the existing procedure, like wordpiece or sentencepiece, or create something by yourself). Here is a simple idea: take 8K most popular words (W), 1K most popular suffixes (S), and 1K most popular prefixes (P). Words in W are its own tokens. Word x outside W should be tokenized as 'p_ _s' where p is the longest prefix of x in P, and s is the longest prefix of W

**C**. write text generation procedure. The procedure should fulfill the following requirements:

1. it should use the RNN language model (trained on sub-word tokens)
2. generated tokens should be presented as a text containing words (without extra spaces, or other extra characters, as begin-of-word introduced during tokenization)
3. all words in a generated text should belond to the corpora (note that this is not guaranteed by LSTM)
4. in generation Top-P sampling should be used (see NN-NLP.6, slide X) 
5. in generated texts every token 3-gram should be uniq
6. *(optionally, +1 point)* all token bigrams in generated texts occur in the corpora

## Task 3

In this task you have to create a network which looks at characters of the word and tries to guess whether the word is a noun, a verb, an adjective, and so on. To be more precise: the input is a word (without context), the output is a POS-tag (Part-of-Speech). Since some words are unambiguous, and we have no context, our network is supposed to return the set of possible tags.

The data is taken from Universal Dependencies English corpus, and of course it contains errors, especially because not all possible tags occured in the data.

Train a network (4p) or two networks (+2p) solving this task. Both networks should look at character n-grams occuring in the word. There are two options:

* **Fixed size:** for instance take 2,3, and 4-character suffixes of the word, use them as  features (whith 1-hot encoding). You can also combine prefix and suffix features. Simple, useful trick: when looking at suffixes, add some '_' characters at the beginning of the word to guarantee that shorter words have suffixes of a desired length.

* **Variable size:** take for instance 4-grams (or 4 grams and 3-grams), use Deep Averaging Network. Simple trick: add extra character at the beginning and at the end of the word, to add the information, that ngram occurs at special position ('ed' at the end has slightly different meaning that 'ed' in the middle)


## Task 4

Apply seq2seq model (you can modify the code from this tutorial: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) to compute grapheme to phoneme conversion for English. Train the model on dev_cmu_dict.txt and test it on test_cmu_dict.txt. Report accuracy of your solution using two metrics:
* exact match (how many words are perfectly converted to phonemes)
* exact match without stress (how many words are perfectly converted to phonemes when we remove the information about stress)
